In [71]:
import pandas as pd

In [72]:
info_df = pd.read_csv('./data/total_info_data.csv')
download_df = pd.read_csv('./data/total_data.csv')

In [5]:
info_df.head()

,area,rank_gs,title,rate_gs,rank_pc,rate_pc,rank_ss,rate_ss,agg_rate,agg_rank,rank,pdf_link,abstract,citationCount,referenceCount,influentialCitationCount,fieldsOfStudy
0,3D Reconstruction,9.0,occupancy networks: learning 3d reconstruction...,5.0,18.0,5.0,1.0,5.0,5.0,10.2,1,http://openaccess.thecvf.com/content_CVPR_2019...,"With the advent of deep neural networks, learn...",623.0,86.0,136.0,['Computer Science']
1,3D Reconstruction,68.0,weakly supervised 3d reconstruction with adver...,4.0,4.0,5.0,29.0,5.0,4.7,30.7,2,https://arxiv.org/pdf/1705.10904,Supervised 3D reconstruction has witnessed a s...,85.0,60.0,3.0,['Computer Science']
2,3D Reconstruction,95.0,infinitam v3: a framework for large-scale 3d r...,4.0,17.0,5.0,32.0,5.0,4.7,44.9,3,https://arxiv.org/pdf/1708.00783,Volumetric models have become a popular repres...,59.0,13.0,6.0,['Computer Science']
3,3D Reconstruction,30.0,pix2vox: context-aware 3d reconstruction from ...,5.0,45.0,4.0,5.0,5.0,4.6,28.5,4,http://openaccess.thecvf.com/content_ICCV_2019...,Recovering the 3D representation of an object ...,70.0,49.0,8.0,['Computer Science']
4,3D Reconstruction,33.0,image2mesh: a learning framework for single im...,5.0,94.0,4.0,15.0,5.0,4.6,52.0,5,https://arxiv.org/pdf/1711.10669,One challenge that remains open in 3D deep lea...,70.0,53.0,3.0,['Computer Science']


In [11]:
download_df = download_df.sort_values(by=['area', 'agg_rate', 'agg_rank'], ascending=[True, False, True]).reset_index(drop=True)
info_df = info_df.sort_values(by=['area', 'agg_rate', 'agg_rank'], ascending=[True, False, True]).reset_index(drop=True)

In [28]:
new_df = download_df.merge(info_df[['area','title', 'abstract', 'citationCount', 'referenceCount', 'influentialCitationCount']], on=['area','title'], how='left', suffixes=['_d','_i'])

In [31]:
new_df['abstract_d'].fillna('',inplace=True)
new_df['abstract_i'].fillna('',inplace=True)

In [34]:
from tqdm import tqdm

In [36]:
lack_df = new_df[new_df['abstract_d'] == '']

In [37]:
for idx, row in tqdm(lack_df.iterrows(),total=len(lack_df)):
    if row['abstract_d'] == '':
        row['abstract_d'] = row['abstract_i']
    new_df.loc[idx] = row

100%|██████████| 151/151 [00:27<00:00,  5.43it/s]


In [40]:
new_df = new_df.drop(columns='abstract_i')

In [43]:
new_df = new_df.rename(columns={'abstract_d':'abstract'})

In [46]:
new_df = new_df.drop(columns=['Unnamed: 0','Unnamed: 0.1', 'Unnamed: 0.1.1'])

In [48]:
new_df = new_df.sort_values(by=['area', 'agg_rate', 'agg_rank'], ascending=[True, False, True]).reset_index(drop=True)
new_df['rank'] = new_df.groupby('area').cumcount()+1

In [58]:
len(info_df)

12623

In [73]:
download_df['abstract'].fillna('', inplace=True)
info_df['abstract'].fillna('', inplace=True)

In [90]:
lack_df = download_df[download_df['abstract'] == '']

In [91]:
for idx, row in tqdm(lack_df.iterrows(),total=len(lack_df)):
    title = row['title']
    ser = info_df[info_df['title'] == title].iloc[0]
    row['absrtact'] = ser['abstract']
    download_df.loc[idx] = row

100%|██████████| 115/115 [00:00<00:00, 238.30it/s]


In [93]:
download_df = download_df[download_df['abstract'] != '']

In [96]:
download_df = download_df.drop(columns=['Unnamed: 0','Unnamed: 0.1','Unnamed: 0.1.1'])

In [108]:
download_df = download_df.drop_duplicates(subset=['area','title'])

In [110]:
download_df.columns

Index(['area', 'rank_gs', 'title', 'rate_gs', 'rank_pc', 'rate_pc', 'rank_ss',
       'rate_ss', 'agg_rate', 'agg_rank', 'rank', 'pdf_link', 'abstract',
       'citationCount', 'referenceCount', 'influentialCitationCount',
       'fieldsOfStudy', 'file_name'],
      dtype='object')

In [111]:
new_df = download_df[['area', 'rank_gs', 'title', 'rate_gs', 'rank_pc', 'rate_pc', 'rank_ss',
       'rate_ss', 'agg_rate', 'agg_rank', 'rank', 'pdf_link', 'abstract',
    'file_name']].merge(info_df[['area','title', 'citationCount', 'referenceCount', 'influentialCitationCount']], on=['area','title'], how='left', suffixes=['_d','_i'])

In [114]:
new_df = new_df.drop_duplicates(subset=['area','title'])

In [119]:
new_df = new_df.sort_values(by=['area', 'agg_rate', 'agg_rank'], ascending=[True, False, True]).reset_index(drop=True)
new_df['rank'] = new_df.groupby('area').cumcount()+1

In [214]:
new_new_df = new_df.drop_duplicates(subset=['area','pdf_link','abstract'])

In [215]:
sum(new_new_df.duplicated('abstract'))

1533

In [150]:
import xmltodict
import requests

In [229]:
lack_abstract_df = new_new_df.loc[new_new_df.duplicated(['area','abstract'])]
drop_idxs = []

In [231]:
lack_abstract_df = lack_abstract_df.drop(drop_idxs)
drop_idxs = []
for idx, paper in tqdm(lack_abstract_df.iterrows(), total = lack_abstract_df.shape[0]):
    title_formatted = paper['title'].replace(' ', '+')
    response = requests.get(f'http://export.arxiv.org/api/query?search_query=ti:{title_formatted}&start=0&max_results=1')
    data = xmltodict.parse(response.text)['feed']
    if 'entry' in data:
        paper['abstract'] = data['entry']['summary'].replace('\n',' ')
    else:
        drop_idxs.append(idx)
    new_new_df.loc[idx] = paper

100%|██████████| 858/858 [09:27<00:00,  1.51it/s]


In [232]:
new_new_df = new_new_df[new_new_df['abstract']!='']

In [234]:
new_new_df = new_new_df.drop_duplicates(['area','abstract'])

In [219]:
lack_link_df = new_new_df[new_new_df.duplicated(['area','pdf_link'])]
drop_idxs = []

In [220]:
lack_link_df = lack_link_df.drop(drop_idxs)
drop_idxs = []
for idx, paper in tqdm(lack_link_df.iterrows(), total = lack_link_df.shape[0]):
    title_formatted = paper['title'].replace(' ', '+')
    response = requests.get(f'http://export.arxiv.org/api/query?search_query=ti:{title_formatted}&start=0&max_results=1')
    data = xmltodict.parse(response.text)['feed']
    if 'entry' in data:
        paper['pdf_link'] = data['entry']['link'][1]['@href']
    else:
        drop_idxs.append(idx)
    new_new_df.loc[idx] = paper
        

  0%|          | 0/420 [00:00<?, ?it/s]/home/junjayz/miniconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1797: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, v, pi)
100%|██████████| 420/420 [05:35<00:00,  1.25it/s]


In [224]:
sum(new_new_df.duplicated('abstract'))

1475

In [222]:
sum(new_new_df.duplicated(['area','pdf_link']))

420

In [223]:
new_new_df = new_new_df.drop_duplicates(['area','pdf_link'])

In [225]:
import time

In [226]:
lack_abstract_df = new_new_df.loc[new_new_df.duplicated(['area', 'abstract'])]
drop_idxs = []

In [227]:
len(lack_abstract_df)

1035

In [228]:
lack_abstract_df.drop(drop_idxs, inplace=True)
drop_idxs = []
for idx, paper in tqdm(lack_abstract_df.iterrows(), total=lack_abstract_df.shape[0]):
    title_formatted = paper['title'].replace(' ', '+')
    data = requests.get(f'https://api.semanticscholar.org/graph/v1/paper/search?query={title_formatted}&limit=1&fields=abstract,referenceCount,citationCount,influentialCitationCount').json()
    if data.get('message') is not None:
        lack_abstract_df = new_new_df.loc[new_new_df.duplicated('abstract')]
        print(data.get('message'))
        time.sleep(120)
    else:
        data = data.get('data')
        if data:
            data = data[0]
            paper['abstract'] = data.get('abstract')
            paper['citationCount'] = data.get('citationCount')
            paper['referenceCount'] = data.get('referenceCount')
            paper['influentialCitationCount'] = data.get('influentialCitationCount')
        else:
            drop_idxs.append(idx)
    new_new_df.loc[idx] = paper
        

/home/junjayz/miniconda3/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
 18%|█▊        | 185/1035 [01:51<08:12,  1.73it/s]

Forbidden


 18%|█▊        | 185/1035 [03:23<15:33,  1.10s/it]


KeyboardInterrupt: 

In [176]:
new_new_df['abstract'].fillna('', inplace=True)

/home/junjayz/miniconda3/lib/python3.9/site-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [237]:
new_new_df['pdf_link'].fillna('', inplace=True)

/home/junjayz/miniconda3/lib/python3.9/site-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [238]:
sum(new_new_df['pdf_link'] == '')

0

In [239]:
sum(new_new_df['abstract'] == '')

0

In [193]:
new_new_df = new_new_df.sort_values(by=['area', 'agg_rate', 'agg_rank'], ascending=[True, False, True]).reset_index(drop=True)
new_new_df['rank'] = new_new_df.groupby('area').cumcount()+1

In [240]:
len(new_new_df['file_name'].unique())

7818

In [243]:
len(new_new_df['title'].unique())

7865

In [246]:
for idx, row in tqdm(new_new_df.iterrows(), total=len(new_new_df)):
    title = row['title']
    area = row['area']
    letters = [word[0:2] for word in title.split(' ')]
    row['file_name'] = area[0]+''.join(letters)
    new_new_df.loc[idx] = row

  0%|          | 0/8255 [00:00<?, ?it/s]/home/junjayz/miniconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1797: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, v, pi)
100%|██████████| 8255/8255 [00:19<00:00, 433.92it/s]


In [247]:
len(new_new_df['title'].unique())

7865

In [248]:
len(new_new_df['file_name'].unique())

8160

In [249]:
new_new_df.to_csv('./data/final_data.csv')

In [250]:
test_df = new_new_df[new_new_df['rank']<=100]

In [251]:
test_df.to_csv('./data/new_test_set.csv')

In [252]:
sum(new_new_df.duplicated('abstract'))

465

In [253]:
sum(new_new_df.duplicated('pdf_link'))

464

In [254]:
new_new_df.groupby('area').count()

,rank_gs,title,rate_gs,rank_pc,rate_pc,rank_ss,rate_ss,agg_rate,agg_rank,rank,pdf_link,abstract,file_name,citationCount,referenceCount,influentialCitationCount
area,,,,,,,,,,,,,,,,
3D Reconstruction,405,405,405,405,405,405,405,405,405,405,405,405,405,405,405,405
Action Recognition,399,399,399,399,399,399,399,399,399,399,399,399,399,399,399,399
Autonomous vehicles,417,417,417,417,417,417,417,417,417,417,417,416,417,417,417,417
Denoising,423,423,423,423,423,423,423,423,423,423,423,423,423,422,422,422
Depth Estimation,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347
Domain adaptation,376,376,376,376,376,376,376,376,376,376,376,376,376,376,376,376
Face Recognition,410,410,410,410,410,410,410,410,410,410,410,410,410,409,409,409
Image Captioning,330,330,330,330,330,330,330,330,330,330,330,330,330,330,330,330
Image Compression,343,343,343,343,343,343,343,343,343,343,343,343,343,343,343,343
